In [ ]:
# The purpose of this notebook is to demonstrate how catchment data such as mean areal precipitation can be explored

In [ ]:
%%capture
!pip install spatialpandas easydev colormap colorcet duckdb dask_geopandas nb_black

In [ ]:
%load_ext lab_black

In [ ]:
import os
import sys

sys.path.insert(0, "../../")
sys.path.insert(0, "../../evaluation/")
# sys.path.insert(0, "../../evaluation/queries/")

from evaluation import utils, config

# import queries  # need to fix path to use original queries
from evaluation.queries import queries

# import dask_geopandas
import duckdb as ddb

In [ ]:
import param
import geopandas as gpd
import pandas as pd
import spatialpandas as spd
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import panel as pn
import datashader as ds
from holoviews.operation.datashader import (
    rasterize,
    shade,
    regrid,
    inspect_points,
    datashade,
    inspect_polygons,
)
from bokeh.models import HoverTool
from holoviews import streams
from shapely.geometry import Point

In [ ]:
hv.extension("bokeh", logo=False)
opts = dict(
    width=700,
    height=500,
)

In [ ]:
def get_catchment_details(
    gdf: gpd.GeoDataFrame, catchment_id: str, reference_time: pd.Timestamp = None
):
    filters = []
    if catchment_id != "all":
        filters.append(
            {"column": "catchment_id", "operator": "like", "value": f"{catchment_id}%"}
        )
    else:
        filters.append({"column": "catchment_id", "operator": "<>", "value": ""})
    if reference_time is not None:
        filters.append(
            {"column": "reference_time", "operator": "=", "value": f"{reference_time}"}
        )

    query = queries.calculate_catchment_metrics(
        config.MEDIUM_RANGE_FORCING_PARQUET,
        config.FORCING_ANALYSIS_ASSIM_PARQUET,
        group_by=["reference_time, catchment_id"],
        order_by=["reference_time, catchment_id"],
        filters=filters,
    )
    df = ddb.query(query).to_df()
    gdf_map = gdf.merge(df, left_on="huc10", right_on="catchment_id")
    gdf_map["name"] = gdf_map["name"].astype("category")
    gdf_map["catchment_id"] = gdf_map["catchment_id"].astype("category")
    spd_map = spd.GeoDataFrame(gdf_map)
    return spd_map

In [ ]:
def get_joined_catchment_timeseries(catchment_id: str, reference_time: pd.Timestamp):
    query = queries.get_joined_catchment_timeseries(
        config.MEDIUM_RANGE_FORCING_PARQUET,
        config.FORCING_ANALYSIS_ASSIM_PARQUET,
        filters=[
            {"column": "reference_time", "operator": "=", "value": f"{reference_time}"},
            {"column": "catchment_id", "operator": "=", "value": catchment_id},
        ],
    )
    df = ddb.query(query).to_df()
    return df

In [ ]:
def get_reference_times():
    query = f"""select distinct(reference_time)as time
        from '{config.MEDIUM_RANGE_FORCING_PARQUET}/*.parquet'
        order by reference_time asc"""
    df = ddb.query(query).to_df()
    times = df.time.tolist()
    return times

In [ ]:
class CatchmentExplorer(param.Parameterized):
    renderer = hv.renderer("bokeh")
    catchments = utils.parquet_to_gdf(config.HUC10_PARQUET_FILEPATH).to_crs("EPSG:3857")
    selected_catchment_id = param.String(default=None, allow_None=True)
    selected_catchment_name = param.String(default=None, allow_None=True)

    reference_times = get_reference_times()
    reference_time = param.ObjectSelector(
        default=reference_times[0], objects=reference_times
    )
    measure = param.ObjectSelector(
        default="bias",
        objects=[
            "bias",
            "max_forecast_delta",
            "observed_variance",
            "forecast_variance",
            "observed_average",
            "forecast_average",
        ],
    )
    huc2 = param.ObjectSelector(
        default="01",
        objects=[
            "all",
            "01",
            "02",
            "03",
            "04",
            "05",
            "06",
            "07",
            "08",
            "09",
            "10",
            "11",
            "12",
            "13",
            "14",
            "15",
            "16",
            "17",
            "18",
        ],
    )

    stream = streams.Tap()

    def get_basemap(self):
        tiles = gv.tile_sources.OSM.opts(**opts)
        return tiles

    def get_catchments(self):
        spd_map = get_catchment_details(self.catchments, self.huc2, self.reference_time)

        catchments = gv.Polygons(
            spd_map,
            crs=ccrs.GOOGLE_MERCATOR,
            vdims=[self.measure, "name", "catchment_id"],
        )

        measure_min = spd_map[self.measure].min()
        measure_max = spd_map[self.measure].max()
        catchments = catchments.redim.range(
            **{f"{self.measure}": (measure_min, measure_max)}
        )

        return catchments

    @param.depends("reference_time", "huc2", "measure")
    def catchment_overlay(self):
        catchments = gv.DynamicMap(self.get_catchments)

        rast_catchments = rasterize(
            catchments,
            aggregator=ds.mean(self.measure),
            precompute=True,
        ).opts(**opts, colorbar=True, cmap="fire")

        tooltips = [
            ("Name", "@name"),
            ("Catchment ID", "@catchment_id"),
            (f"{self.measure}", f"@{self.measure}"),
        ]
        hover_tool = HoverTool(tooltips=tooltips)
        hover = (
            inspect_polygons(rast_catchments)
            .opts(fill_color="yellow", tools=[hover_tool, "tap"])
            .opts(alpha=0.5)
        )

        self.stream.source = rast_catchments

        return hv.DynamicMap(self.get_basemap) * rast_catchments * hover

    @param.depends("stream.x", "stream.y")
    def set_selected_catchment(self):
        pnt = Point(self.stream.x, self.stream.y)
        selected_catchment = self.catchments[(self.catchments.contains(pnt) == True)]
        catchment_id = selected_catchment["huc10"].iloc[0]
        # catchment_name = selected_catchment["name"].iloc[0]

        self.selected_catchment_id = catchment_id
        # self.selected_catchment_name = catchment_name

    @param.depends("selected_catchment_id")
    def table(self):
        if self.selected_catchment_id is None:
            return pn.pane.Str("Select a basin to see plots")
        # df = get_joined_catchment_timeseries(self.catchment_id, self.reference_time)[
        #     ["value_time", "forecast_value", "observed_value"]
        # ]
        # return hv.Table(df, width=700).relabel(
        #     f"{self.catchment_name} | {self.catchment_id} | {self.reference_time}"
        # )
        return pn.pane.Str(self.selected_catchment_id)

    @param.depends("selected_catchment_id")
    def plot(self):
        if self.selected_catchment_id is None:
            return pn.pane.Str("Select a basin to see plots")
        #         df = get_joined_catchment_timeseries(self.catchment_id, self.reference_time)[
        #             ["value_time", "forecast_value", "observed_value"]
        #         ]
        #         forecast_val = hv.Curve(
        #             df, "value_time", "forecast_value", label="forecast_value"
        #         )
        #         forecast_val.opts(tools=["hover"], color="orange")
        #         observed_val = hv.Curve(
        #             df, "value_time", "observed_value", label="observed_value"
        #         )
        #         observed_val.opts(tools=["hover"], color="blue")

        #         viz = (forecast_val * observed_val).relabel(
        #             f"{catchment_name} | {catchment_id} | {reference_time}"
        #         )
        #         return pn.panel(viz, width=700)
        return pn.pane.Str(self.selected_catchment_id)

In [ ]:
catchment_explorer = CatchmentExplorer(name="Catchement Explorer")

layout = pn.Column(
    pn.Row(
        pn.pane.PNG(
            "https://ciroh.ua.edu/wp-content/uploads/2022/08/CIROHLogo_200x200.png",
            width=100,
        ),
        pn.pane.Markdown(
            """
            # CIROH Integrated Evaluation and Exploration System
            ## After-action Analysis
        
        """,
            width=800,
        ),
    ),
    pn.Row(
        pn.Column(
            pn.Param(
                catchment_explorer.param,
                widgets={"reference_time": pn.widgets.DiscretePlayer},
                sizing_mode="stretch_width",
            ),
            # catchment_explorer.param.measure,
            # catchment_explorer.param.huc2,
        ),
        pn.Column(
            pn.Row(pn.panel(catchment_explorer.catchment_overlay)),
            pn.Row(
                pn.Tabs(
                    ("Plot", pn.panel(catchment_explorer.plot)),
                    ("Table", pn.panel(catchment_explorer.table)),
                )
            ),
        ),
    ),
).servable()

layout

In [ ]:
catchment_explorer.selected_catchment_name